### Pre-Configuring Graph Server Credentials

Update the following variables with your actual database credentials: username, password, host, port, and service name to establish a connection.

In [ ]:
from pypgx import setloglevel
from oraclegraph import GraphVisualization
from oraclegraph import PgxGraphVisualization
from opg4py import graph_server

username = 'your_username'
# IMPORTANT: do not put passwords into notebooks in plaintext.
# Instead, fetch it from a secure password store or if given by the user,
# use the password widget: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#password
password = 'your_password'
base_url = 'https://xxx.xx.xx.xx:7007'

setloglevel('ROOT', 'ERROR')

Please check this link for data, table creation https://github.com/oracle-samples/oracle-graph/tree/master/23c-graph-demos.

### PgxGraphVisualization

The following code block tests PgxGraphVisualization features provided by the Jupyter extension.

**Note:** This page shows how to use a PGX server with VisualQuery to get visualization first and then assign it to GraphVisualization.data.


In [ ]:
from pypgx import setloglevel
from oraclegraph import GraphVisualization
from oraclegraph import PgxGraphVisualization
from opg4py import graph_server

instance = graph_server.get_instance(base_url, username, password)
session = instance.create_session('jupyter')

loaded_graph = session.read_graph_by_name('BANK_GRAPH_PGQL', 'pg_pgql');
# loaded_graph = session.read_graph_by_name('BANK_GRAPH', 'pg_sql');

analyst = session.create_analyst()

# use session+analyst to run algorithms, queries, etc..
pagerank = analyst.pagerank(loaded_graph, rank='pagerank')

result_set = loaded_graph.query_pgql(
    "SELECT x, x.pagerank MATCH (x)"
    " ORDER BY x.pagerank DESC LIMIT 10"
   )
result_set.print()

In [ ]:
props={'size': 20} 
vq = PgxGraphVisualization(session=session)

rule_based_styles = [
    {
        "component": "vertex",
        "target": "vertex",
        "conditions": {
            "conditions": [
                {
                    "property": "NAME",
                    "operator": "!=",
                    "value": ""
                }
            ]
        },
        "modifierStyles": {
            "hover": {
                "label": {
                    "text": "${properties.NAME}"
                }
            }
        },
        "legendTitle": "Hover - Name Caption"
    },
    {
        "component": "vertex",
        "target": "vertex",
        "conditions": {
            "conditions": [
                {
                    "property": "BALANCE",
                    "operator": ">=",
                    "value": 150000
                }
            ]
        },
        "style": {
            "color": "#AA5F71"
        },
        "legendTitle": "Balance >= $150,000"
    },
    {
        "component": "vertex",
        "target": "outgoing",
        "conditions": {
            "conditions": [
                {
                    "property": "BALANCE",
                    "operator": ">=",
                    "value": 150000
                }
            ]
        },
        "animations": [
            [
                {
                    "id": "animation_1",  # Corrected typo from "animiation_1" for clarity
                    "duration": 5,
                    "timingFunction": "linear",
                    "keyFrames": [
                        {
                            "percentage": 100,
                            "style": {
                                "strokeDashoffset": 100
                            }
                        }
                    ]
                }
            ]
        ],
        "style": {
            "color": "#825BBD",
            "dasharray": "dashed3"
        },
        "legendTitle": "Edge Animation"
    }
]
settings = {
    'ruleBasedStyles': rule_based_styles
}
 

query = '''
SELECT *
FROM MATCH (v1)-[e]->(v2) ON {graph_name}
'''.format(graph_name=loaded_graph.name)


json = vq.visualize_query(query)
graph = GraphVisualization(data=json, settings=settings)
graph
